In [19]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pymor.basic import *
from problems import whole_problem
from discretizer import discretize_instationary_IP
from pymor.parameters.base import ParameterSpace

# general options           
N = 10                                                                        # FE Dofs = (N+1)^2                                                
tol = 1e-14                                                                    # safeguard tolerance
tau = 3.5                                                                      # discrepancy parameter
k_max = 50                                                                     # maxit
noise_level = 1e-5        

# choose optimization methods
opt_methods = {     
              "Qr IRGNM",
              "FOM IRGNM",
              "Qr-Vr TR IRGNM",
               }

# choose norm in the parameter space
norm_type = 'L2'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
print('Construct problem..')                                                     
analytical_problem, q_exact, N, problem_type, exact_analytical_problem, energy_problem = whole_problem(
                                                        N = N,
                                                        parameter_location = 'reaction',
                                                        boundary_conditions = 'dirichlet',
                                                        exact_parameter = 'Kirchner',
                                                       )

Construct problem..


In [21]:
diameter = np.sqrt(2)/N
grid_type = RectGrid
low = 0.001                                                                     # lower bound value
up = 1e20                                                                       # upper bound value 
par_dim = (N+1)**2                                                              # dimension of parameter space
q_low = low*np.ones((par_dim,))                                                 # lower bound
q_up =  up*np.ones((par_dim,))                                                  # upper bound
bounds = [q_low, q_up]
q0 = 3*np.ones((par_dim,))                                                      # initial guess for IRGNM
q_0 =  3*np.ones((par_dim,))                                                    # regularization center for IRGNM
opt_data = {'FE_dim': (N+1)**2,                                                 # FE dofs
            'par_dim': par_dim,                                                     
            'noise_level': noise_level,
            'q_exact': q_exact,
            'q0': q0,
            'q_0': q_0, 
            'low': low,
            'up': up,
            'noise_percentage': None,   
            'problem_type': problem_type,
            'q_in_h1': True,
            'norm_on_q': norm_type,
            'B_assemble': False,                                                # options to preassemble affine components or not
            'bounds': bounds
            } 

# create parameter space
parameter_space = ParameterSpace(analytical_problem.parameters, [low,up]) 


In [22]:
print('Discretizing problem...')                                                
primal_fom = discretize_instationary_IP(analytical_problem,
                           diameter = diameter,
                           opt_data = opt_data,
                           exact_analytical_problem = exact_analytical_problem,
                           energy_product_problem = energy_problem,
                           grid_type = grid_type
                           )


Discretizing problem...


Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [17]:
print("A")

A


In [30]:
analytical_problem.stationary_part.diffusion

ConstantFunction(dim_domain=2)

In [37]:
q = primal_fom.operator.parameters.parse(np.zeros(121))
primal_fom.operator.assemble(q)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

NumpyMatrixOperator(<121x121 sparse, 769 nnz>, source_id='STATE', range_id='STATE')

In [35]:
q

Mu({'diffusion': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])})

In [36]:
primal_fom.operator.coefficients[2].

ProjectionParameterFunctional('diffusion', size=121, index=0, name='diffusion_0_0')